# Adding emulators

In addition to providing a library of core emulators, AutoEmulate is designed to be easily extensible. This tutorial walks you through the steps of adding new emulators to the library. We cover two scenarios: adding new Gaussian Process kernels and adding entirely new models.

## 1. Adding Gaussian Process kernels

Gaussian Processes (GPs) are primarily defined by their kernel functions, which determine the covariance structure of the data. AutoEmulate includes several built-in GP kernels:
- Radial Basis Function (RBF)
- Matern 3/2
- Matern 5/2
- Rational Quadratic (RQ)
- Linear

You can easily create new kernels by composing any two or more of these existing kernels. For example, you might want to create a kernel that combines the RBF and Linear kernels to capture both smooth variations and linear trends in your data.

In AutoEmulate, each kernel is defined by an initialisation function that takes as inputs the number of data input features and the number of output features. Below we define a custom kernel function following this pattern.

In [ ]:
from autoemulate.emulators.gaussian_process.kernel import rbf_kernel, linear_kernel

def rbs_plus_linear_kernel(n_features, n_outputs):
    """
    Example of a custom kernel function that combines RBF and linear kernels.
    """
    return rbf_kernel(n_features, n_outputs) + linear_kernel(n_features, n_outputs)

Once this function has been defined, you can create a new GP emulator class using the `create_gp_subclass` function.

In [ ]:
from autoemulate.emulators.gaussian_process.exact import GaussianProcess, create_gp_subclass

GaussianProcessRBFandLinear = create_gp_subclass(
   "GaussianProcessRBFandLinear", 
   GaussianProcess, 
   # the custom kernel function goes here
   covar_module_fn=rbs_plus_linear_kernel,
)

Now we can tell AutoEmulate to use the new GP class by passing it to the `models` argument when initialising an `AutoEmulate` object.

In [ ]:
from autoemulate import AutoEmulate
import torch

# create some example data
x = torch.linspace(0, 1, 100).unsqueeze(-1)
y = torch.sin(2 * 3.14 * x) + 0.1 * torch.randn_like(x)

ae = AutoEmulate(x, y, models=[GaussianProcessRBFandLinear])

In [ ]:
ae.summarise()